# 前処理

In [1]:
import dask
import re
import dask
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import datetime

#air
#reserve
df_air_reserve=pd.read_csv('~/Desktop/ml/restaurant_vistor/air_reserve.csv')
df_air_reserve['visit_date']=df_air_reserve['visit_datetime'].str.split().str[0]
df_air_reserve['visit_time']=df_air_reserve['visit_datetime'].str.split().str[1]
df_air_reserve['reserve_date']=df_air_reserve['reserve_datetime'].str.split().str[0]
df_air_reserve['reserve_time']=df_air_reserve['reserve_datetime'].str.split().str[1]
# group by store_id & visit_date
df_air_reserve_grouped=df_air_reserve.groupby(['air_store_id','visit_date']).sum()
df_air_reserve_grouped=df_air_reserve_grouped.reset_index()

#store info
df_air_store_info=pd.read_csv('~/Desktop/ml/restaurant_vistor/air_store_info.csv')

#visit
df_air_visit_data=pd.read_csv('~/Desktop/ml/restaurant_vistor/air_visit_data.csv')

#date info
df_date_info=pd.read_csv('~/Desktop/ml/restaurant_vistor/date_info.csv')

# join tables df_air_reserve_grouped, df_air_store_info & df_air_visit_data
df_air=df_air_reserve_grouped.merge(df_air_store_info, left_on='air_store_id', right_on='air_store_id', how='outer')
df_air=df_air.merge(df_air_visit_data, left_on=['air_store_id', 'visit_date'], right_on=['air_store_id', 'visit_date'], how='outer')

# join date_info
df_air=df_air.merge(df_date_info, left_on=['visit_date'], right_on=['calendar_date'], how='left')

df_air.drop(columns='calendar_date', inplace=True)

#hpg
# reserve & visit data
df_hpg_reserve=pd.read_csv('~/Desktop/ml/restaurant_vistor/hpg_reserve.csv')
df_hpg_reserve['visit_date']=df_hpg_reserve['visit_datetime'].str.split().str[0]
df_hpg_reserve['visit_time']=df_hpg_reserve['visit_datetime'].str.split().str[1]
df_hpg_reserve['reserve_date']=df_hpg_reserve['reserve_datetime'].str.split().str[0]
df_hpg_reserve['reserve_time']=df_hpg_reserve['reserve_datetime'].str.split().str[1]
# group by store_id & visit_date
df_hpg_reserve_grouped=df_hpg_reserve.groupby(['hpg_store_id', 'visit_date']).sum()
df_hpg_reserve_grouped=df_hpg_reserve_grouped.reset_index()

#store info
df_hpg_store_info=pd.read_csv('~/Desktop/ml/restaurant_vistor/hpg_store_info.csv')

# join tables df_hpg_reserve & df_hpg_store_info
df_hpg=df_hpg_reserve_grouped.merge(df_hpg_store_info, on='hpg_store_id', how='outer')

# join air & hpg
df_store_id_relation=pd.read_csv('~/Desktop/ml/restaurant_vistor/store_id_relation.csv')
df=df_air.merge(df_store_id_relation, on='air_store_id', how='left')
df=df.merge(df_hpg, on=['hpg_store_id', 'visit_date'], how='left')

# drop unecessary columns
df.drop(columns=['reserve_visitors_x', 'reserve_visitors_y', 'latitude_x', 'latitude_y', 'longitude_x', 'longitude_y'], inplace=True)
df.drop(columns=['hpg_store_id', 'hpg_genre_name', 'hpg_area_name'], inplace=True)

# change date format to datetime
df['visit_date']=pd.to_datetime(df['visit_date'])

In [15]:
df.head(5)

,air_store_id,visit_date,air_genre_name,air_area_name,visitors,day_of_week,holiday_flg
0,air_00a91d42b08b08d9,2016-10-31,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,26.0,Monday,0.0
1,air_00a91d42b08b08d9,2016-12-05,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,33.0,Monday,0.0
2,air_00a91d42b08b08d9,2016-12-14,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,30.0,Wednesday,0.0
3,air_00a91d42b08b08d9,2016-12-17,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,17.0,Saturday,0.0
4,air_00a91d42b08b08d9,2016-12-20,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,37.0,Tuesday,0.0


# time series framework
### colection period: 4 weeks
### validation periad: 1 week
### move window by 1 week

In [30]:
col_s=datetime.datetime(2016, 7, 1)
last_col_s=datetime.datetime(2016,11,25)

while(col_s<=last_col_s):
    end_date = col_s + datetime.timedelta(days=10)
    col_e=col_s + datetime.timedelta(days=27)
    val_s=col_s + datetime.timedelta(days=28)
    val_e=col_s + datetime.timedelta(days=34)

    print(col_s, col_e, val_s, val_e)
    col_s=col_s + datetime.timedelta(days=7)

2016-07-01 00:00:00 2016-07-28 00:00:00 2016-07-29 00:00:00 2016-08-04 00:00:00
2016-07-08 00:00:00 2016-08-04 00:00:00 2016-08-05 00:00:00 2016-08-11 00:00:00
2016-07-15 00:00:00 2016-08-11 00:00:00 2016-08-12 00:00:00 2016-08-18 00:00:00
2016-07-22 00:00:00 2016-08-18 00:00:00 2016-08-19 00:00:00 2016-08-25 00:00:00
2016-07-29 00:00:00 2016-08-25 00:00:00 2016-08-26 00:00:00 2016-09-01 00:00:00
2016-08-05 00:00:00 2016-09-01 00:00:00 2016-09-02 00:00:00 2016-09-08 00:00:00
2016-08-12 00:00:00 2016-09-08 00:00:00 2016-09-09 00:00:00 2016-09-15 00:00:00
2016-08-19 00:00:00 2016-09-15 00:00:00 2016-09-16 00:00:00 2016-09-22 00:00:00
2016-08-26 00:00:00 2016-09-22 00:00:00 2016-09-23 00:00:00 2016-09-29 00:00:00
2016-09-02 00:00:00 2016-09-29 00:00:00 2016-09-30 00:00:00 2016-10-06 00:00:00
2016-09-09 00:00:00 2016-10-06 00:00:00 2016-10-07 00:00:00 2016-10-13 00:00:00
2016-09-16 00:00:00 2016-10-13 00:00:00 2016-10-14 00:00:00 2016-10-20 00:00:00
2016-09-23 00:00:00 2016-10-20 00:00:00 

# prepare learning and validation data

In [25]:
# example: first loop
col_s=datetime.datetime(2016, 7, 1)
col_e=col_s + datetime.timedelta(days=27)
val_s=col_s + datetime.timedelta(days=28)
val_e=col_s + datetime.timedelta(days=34)

df_learn=df[(col_s<=df['visit_date']) & (df['visit_date']<=col_e)]
df_validate=df[(val_s<=df['visit_date']) & (df['visit_date']<=val_e)]

In [29]:
print(col_s, col_e, val_s, val_e)

2016-07-01 00:00:00 2016-07-28 00:00:00 2016-07-29 00:00:00 2016-08-04 00:00:00


In [27]:
df_learn.head(10)

,air_store_id,visit_date,air_genre_name,air_area_name,visitors,day_of_week,holiday_flg
1017,air_0a74a5408a0b8642,2016-07-01,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,22.0,Friday,0.0
1018,air_0a74a5408a0b8642,2016-07-02,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,34.0,Saturday,0.0
1019,air_0a74a5408a0b8642,2016-07-04,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,39.0,Monday,0.0
1020,air_0a74a5408a0b8642,2016-07-05,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,27.0,Tuesday,0.0
1021,air_0a74a5408a0b8642,2016-07-06,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,2.0,Wednesday,0.0
1022,air_0a74a5408a0b8642,2016-07-07,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,34.0,Thursday,0.0
1023,air_0a74a5408a0b8642,2016-07-08,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,11.0,Friday,0.0
1024,air_0a74a5408a0b8642,2016-07-09,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,48.0,Saturday,0.0
1025,air_0a74a5408a0b8642,2016-07-11,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,31.0,Monday,0.0
1026,air_0a74a5408a0b8642,2016-07-12,Western food,Hokkaidō Sapporo-shi Minami 3 Jōnishi,18.0,Tuesday,0.0


In [28]:
df_validate.head(10)

,air_store_id,visit_date,air_genre_name,air_area_name,visitors,day_of_week,holiday_flg
22539,air_bb4ff06cd661ee9b,2016-07-31,Japanese food,Tōkyō-to Shinagawa-ku Hiromachi,NaN,Sunday,0.0
30508,air_ba937bf13d40fb24,2016-07-29,NaN,NaN,33.0,Friday,0.0
30509,air_ba937bf13d40fb24,2016-07-30,NaN,NaN,21.0,Saturday,0.0
30510,air_ba937bf13d40fb24,2016-08-01,NaN,NaN,13.0,Monday,0.0
30511,air_ba937bf13d40fb24,2016-08-02,NaN,NaN,12.0,Tuesday,0.0
30512,air_ba937bf13d40fb24,2016-08-03,NaN,NaN,27.0,Wednesday,0.0
30513,air_ba937bf13d40fb24,2016-08-04,NaN,NaN,24.0,Thursday,0.0
30894,air_25e9888d30b386df,2016-07-29,NaN,NaN,9.0,Friday,0.0
30895,air_25e9888d30b386df,2016-07-30,NaN,NaN,4.0,Saturday,0.0
30896,air_25e9888d30b386df,2016-07-31,NaN,NaN,1.0,Sunday,0.0
